
*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher & Ali Ahmad Khan<br>
**Date:**  15.11.2023

--- 

# Exploring the counting data

This script loads the 'preprocessed_lhm_counting_data.parquet','preprocessed_bast_counting_data.parquet' files, cocatenates them and then adds road_types from 'visum_links.gpkg' and day_types from Calender Module. Keeps rows with daily_value consistent with sum of hourly values(error_bound=2%).Adds a comppleteness check for road links. Adds an outlier check based on data grouped on road_link, vehicle_class and day_type for daily_value. Stores the file in parquet format

**Required steps**
- Import Preprocessed files and convert columns to meaningful datatypes
- Import visum package and Calender Module respectively.




In [1]:
import sys

import pandas as pd
import numpy as np
import geopandas as gpd


from bokeh.palettes import Category10
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

# import custom modules
sys.path.append('../utils')
from excel_calendar import Calendar
import data_paths


Loading BokehJS ...

# Import and prepare datasets

In [2]:
# import counting data
mst_file_path = data_paths.MST_COUNTING_PATH + 'preprocessed_lhm_counting_data.parquet'
bast_file_path = data_paths.BAST_COUNTING_PATH + 'preprocessed_bast_counting_data.parquet'

counting_data = pd.concat([pd.read_parquet(mst_file_path),
                           pd.read_parquet(bast_file_path)], axis=0)

# import visum_links data 
visum_file_path = data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg'
visum = gpd.read_file(visum_file_path)

# import calender
cal_obj = Calendar()

### Aggregate single detector counts and divide in "volume" and "speed" dataset

In [3]:
# aggregate and reduce to volume dataset
volume  = counting_data.groupby(['metric','road_link_id', 
                               'vehicle_class','date']).sum(numeric_only = True).loc['volume']
speed = counting_data.groupby(['metric','road_link_id', 
                               'vehicle_class','date']).mean(numeric_only = True).loc['speed']
volume = volume.drop(['detector_id'], axis =1)
volume = volume.reset_index()
speed = speed.reset_index()

### Append road type information

In [4]:
# insert road type
road_types = visum.set_index('road_link_id')['road_type'].to_dict()
volume.insert(4,'road_type' , volume['road_link_id'].map(road_types))

### Append day type information

In [5]:
dates = volume['date'].unique()
day_types = {date:cal_obj.get_day_type(date) for date in dates}
volume.insert(5, 'day_type', volume['date'].map(day_types))

# Cleaning of the counting data

In [6]:
# only take data into account if daily value >1 and < 40000

# sum of all hour values of the day needs to be consistent with the sum of the day -> error_bound e=2%
e = 0.02

# Filter rows based on the condition
volume_processed = volume[
    volume.iloc[:, 6:].sum(axis=1).between(
        volume['daily_value'] * (1 - e),
        volume['daily_value'] * (1 + e)
    )
].copy()
volume_processed.reset_index(inplace=True)
# how to remove the outliers?
# Filter rows where 'date' is between 2019-01-01 and 2023-12-31

## Data Completeness Check

- Add checks called is_complete which is True when road link has data starting from 2018 and at the earliest ending in 2022 else False

In [7]:
# Define the date range
start_date = pd.Timestamp('2018-12-31')
end_date = pd.Timestamp('2022-01-01')

# Group the DataFrame by 'road_link_id' and filter based on the date range
filtered_groups_df = volume_processed.groupby('road_link_id').filter(
    lambda x: (x['date'].min() <= start_date) and (x['date'].max() >= end_date)
)
""
# Create a new column 'is_complete' and set it to True for rows in filtered_groups
volume_processed['is_complete'] = volume_processed.index.isin(filtered_groups_df.index)

### Visualisation of a Road Link's Daily Value

In [8]:
#road_link = 1.28950000e+04
#road_link = 5.62782535e+08
#road_link = 9.02000000e+02
#road_link = 1.23500000e+04
#road_link = 2723.0
#road_link = 1.82410000e+04
road_link = 2147483452

# Filter the DataFrame for the specific road link and 'PC' vehicle class
filtered_data = volume_processed[(volume_processed['road_link_id'] == road_link) & (volume_processed['vehicle_class'] == 'SNF')]

# Create a Bokeh figure
p = figure(x_axis_type="datetime", width=800, height=350)

# Create a ColumnDataSource from the filtered data
source = ColumnDataSource(filtered_data)

# Plot a line chart
p.line('date', 'daily_value', source=source,)

# Show the plot
show(p)


## Outlier Detection

In [9]:
# Define the columns for grouping and the column for outlier detection
group_columns = ['road_link_id', 'vehicle_class', 'day_type']
daily_values_column = 'daily_value'

# Initialize a new DataFrame to store the outliers
volume_processed_outliers_df = pd.DataFrame(columns=volume_processed.columns)

# Define the threshold for modified Z-Score outliers (adjust as needed)
modified_z_score_threshold = 3.5

def detect_outliers(group):
    daily_values = group[daily_values_column]
    median = np.median(daily_values)
    mad = np.median(np.abs(daily_values - median))
    modified_z_scores = np.abs(0.6745 * (daily_values - median) / mad)
    group['is_outlier'] = modified_z_scores > modified_z_score_threshold
    return group

# Apply the outlier detection function to each group and concatenate the results
volume_processed_outliers_df = volume_processed.groupby(group_columns).apply(detect_outliers)

# Reset the index and rename axis to obtain a flat column hierarchy
volume_processed_outliers_df = volume_processed_outliers_df.reset_index(drop=True)

### Visualisation of a Road Link's Daily Value Groupbed on Day Type

#### Without Outlier Removal

In [10]:
# volume_processed_outliers_df[(volume_processed_outliers_df['road_link_id'] == road_link) & (volume_processed_outliers_df['vehicle_class'] == 'SNF')].groupby('day_type')['daily_value'].plot(kind='line', xlabel='date',ylabel='daily_value',legend=True)

# Filter and Groupby the data according to your requirements
groups = volume_processed_outliers_df[(volume_processed_outliers_df['road_link_id'] == road_link) & (volume_processed_outliers_df['vehicle_class'] == 'SNF')].groupby('day_type')

# Initialise and fill a Data Dictonary of Lists for date, daily_value and day_type as legend
data = {'date': [], 'daily_value': [], 'legend': []}
for i, df in groups:
    data['date'].append(df.date.tolist())
    data['daily_value'].append(df.daily_value.tolist())
    data['legend'].append(i)
data['color'] = Category10[10][0: len(groups)]


# interaction tools for the bokeh figure
other_tools = "pan, box_zoom, save, reset, help,"

# Create a Bokeh figure
p = figure(x_axis_type = "datetime", tools="xwheel_zoom,"+other_tools)

# plot a multi line chart
p.multi_line(xs = 'date',
             ys = 'daily_value',
             line_color = 'color',
             legend_field = 'legend',
             source = data)

# Show the plot
show(p)

#### With Outlier Removal

In [11]:
# volume_processed_outliers_df[(volume_processed_outliers_df['road_link_id'] == road_link) & (volume_processed_outliers_df['vehicle_class'] == 'SNF') & (volume_processed_outliers_df['is_outlier'] == False)].groupby('day_type')['daily_value'].plot(kind='line', xlabel='date',ylabel='daily_value',legend=True)

# Filter and Groupby the data according to your requirements
groups = volume_processed_outliers_df[(volume_processed_outliers_df['road_link_id'] == road_link) & (volume_processed_outliers_df['vehicle_class'] == 'SNF') & (volume_processed_outliers_df['is_outlier'] == False)].groupby('day_type')

# Initialise and fill a Data Dictonary of Lists for date, daily_value and day_type as legend
data = {'date': [], 'daily_value': [], 'legend': []}
for i, df in groups:
    data['date'].append(df.date.tolist())
    data['daily_value'].append(df.daily_value.tolist())
    data['legend'].append(i)
data['color'] = Category10[10][0: len(groups)]

# interaction tools for the bokeh figure
other_tools = "pan, box_zoom, save, reset, help,"

# Create a Bokeh figure
p = figure(x_axis_type = "datetime", tools="xwheel_zoom,"+other_tools)

# plot a multi line chart
p.multi_line(xs = 'date',
             ys = 'daily_value',
             line_color = 'color',
             legend_field = 'legend',
             source = data)

# Show the plot
show(p)

## Store as Parquet File

In [12]:
# path to mst counting data
data_path = data_paths.COUNTING_PATH

# Convert all columns to string to parquet storage
volume_processed_outliers_df.columns = volume_processed_outliers_df.columns.astype(str)

# Store the dataframe as a parquet file
volume_processed_outliers_df.to_parquet(data_path+'preprocessed_bast_counting_data.parquet', index=False)

In [13]:
volume_processed_outliers_df

,index,road_link_id,vehicle_class,date,daily_value,road_type,day_type,00:00-01:00,01:00-02:00,02:00-03:00,...,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-24:00,is_complete,is_outlier
0,8,0.000000e+00,BUS,2018-01-09,54.0,NaN,0,1.0,0.0,1.0,...,1.0,5.0,2.0,1.0,4.0,0.0,0.0,0.0,True,True
1,9,0.000000e+00,BUS,2018-01-10,40.0,NaN,0,0.0,2.0,0.0,...,6.0,5.0,1.0,0.0,2.0,2.0,1.0,0.0,True,True
2,10,0.000000e+00,BUS,2018-01-11,67.0,NaN,0,0.0,0.0,0.0,...,4.0,5.0,2.0,3.0,4.0,2.0,0.0,0.0,True,True
3,15,0.000000e+00,BUS,2018-01-16,89.0,NaN,0,1.0,0.0,0.0,...,6.0,6.0,4.0,6.0,2.0,5.0,3.0,4.0,True,True
4,16,0.000000e+00,BUS,2018-01-17,67.0,NaN,0,1.0,0.0,0.0,...,4.0,1.0,3.0,0.0,2.0,2.0,0.0,0.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960382,987728,2.147483e+09,SNF,2022-12-04,79.0,Local/Collector,4,2.0,2.0,2.0,...,4.0,2.0,4.0,3.0,6.0,4.0,6.0,5.0,True,False
960383,987735,2.147483e+09,SNF,2022-12-11,79.0,Local/Collector,4,1.0,2.0,3.0,...,6.0,3.0,2.0,5.0,2.0,3.0,9.0,6.0,True,False
960384,987742,2.147483e+09,SNF,2022-12-18,69.0,Local/Collector,4,0.0,3.0,3.0,...,3.0,5.0,1.0,1.0,2.0,3.0,3.0,3.0,True,False
960385,987749,2.147483e+09,SNF,2022-12-25,25.0,Local/Collector,4,0.0,3.0,0.0,...,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,True,False
